In [0]:
storage_account_name = "mega2003storage"
container_name = "megacontainer1"
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-12-08T21:33:57Z&st=2024-12-08T13:33:57Z&spr=https&sig=ypRx6ESU6fE9kYORkJ10AGKc7giY6w4uNBkGRuHOA5w%3D"
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point = "/mnt/megacontainer1",  
    extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token}
)


True

In [0]:
file_path = "/mnt/megacontainer1/insurance.csv"
insurance_df = spark.read.csv(file_path, header=True, inferSchema=True)
insurance_df.show()


+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

In [0]:
insurance_df.printSchema()


root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [0]:
insurance_df.describe().show()



+-------+------------------+------+------------------+------------------+------+---------+------------------+
|summary|               age|   sex|               bmi|          children|smoker|   region|           charges|
+-------+------------------+------+------------------+------------------+------+---------+------------------+
|  count|              1338|  1338|              1338|              1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  NULL|30.663396860986538|   1.0949177877429|  NULL|     NULL|13270.422265141257|
| stddev|14.049960379216172|  NULL| 6.098186911679017|1.2054927397819095|  NULL|     NULL|12110.011236693996|
|    min|                18|female|             15.96|                 0|    no|northeast|         1121.8739|
|    max|                64|  male|             53.13|                 5|   yes|southwest|       63770.42801|
+-------+------------------+------+------------------+------------------+------+---------+------------------+



In [0]:
insurance_df = insurance_df.dropna()


In [0]:
from pyspark.sql.functions import col, sum as _sum
null_counts = insurance_df.select([(col(c).isNull().cast("int")).alias(c) for c in insurance_df.columns])
null_counts_agg = null_counts.agg(*[ _sum(col(c)).alias(c) for c in null_counts.columns])
null_counts_agg.show()

+---+---+---+--------+------+------+-------+
|age|sex|bmi|children|smoker|region|charges|
+---+---+---+--------+------+------+-------+
|  0|  0|  0|       0|     0|     0|      0|
+---+---+---+--------+------+------+-------+



In [0]:
from pyspark.sql.functions import mean
for column in insurance_df.columns:
    if dict(insurance_df.dtypes)[column] in ['int', 'double', 'float']:
        mean_value = insurance_df.select(mean(col(column))).first()[0]
        insurance_df = insurance_df.fillna({column: mean_value})


In [0]:
from pyspark.sql.functions import mean

for column in insurance_df.columns:
    if dict(insurance_df.dtypes)[column] in ['int', 'double', 'float']:
        mean_value = insurance_df.select(mean(col(column))).first()[0]
        if mean_value is not None: 
            insurance_df = insurance_df.fillna({column: mean_value})
        else:
            insurance_df = insurance_df.fillna({column: 0})


In [0]:
insurance_df.describe().show()
insurance_df.groupBy("smoker").mean("charges").show()
insurance_df.groupBy("sex").mean("charges").show()


+-------+------------------+------+------------------+------------------+------+---------+------------------+
|summary|               age|   sex|               bmi|          children|smoker|   region|           charges|
+-------+------------------+------+------------------+------------------+------+---------+------------------+
|  count|              1338|  1338|              1338|              1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  NULL|30.663396860986538|   1.0949177877429|  NULL|     NULL|13270.422265141257|
| stddev|14.049960379216172|  NULL| 6.098186911679017|1.2054927397819095|  NULL|     NULL|12110.011236693996|
|    min|                18|female|             15.96|                 0|    no|northeast|         1121.8739|
|    max|                64|  male|             53.13|                 5|   yes|southwest|       63770.42801|
+-------+------------------+------+------------------+------------------+------+---------+------------------+

+------+-

In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

indexer = StringIndexer(inputCols=["sex", "smoker", "region"], outputCols=["sex_index", "smoker_index", "region_index"])
encoder = OneHotEncoder(inputCols=["sex_index", "smoker_index", "region_index"], outputCols=["sex_vec", "smoker_vec", "region_vec"])
assembler = VectorAssembler(inputCols=["age", "bmi", "children", "sex_vec", "smoker_vec", "region_vec"], outputCol="features")
pipeline = Pipeline(stages=[indexer, encoder, assembler])
insurance_preprocessed = pipeline.fit(insurance_df).transform(insurance_df)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
from pyspark.sql import functions as F
train_data, test_data = insurance_preprocessed.randomSplit([0.8, 0.2], seed=42)

In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="charges")
lr_model = lr.fit(train_data)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

In [0]:
predictions = lr_model.transform(test_data)
predictions.select("charges", "prediction").show()


+-----------+-------------------+
|    charges|         prediction|
+-----------+-------------------+
|  2201.0971|  308.0785935124077|
| 18223.4512| 26033.038976080497|
|7323.734819| 2741.4005389676313|
|  1622.1885| 2385.8281368435764|
| 2211.13075|  5058.635876565404|
|  1631.6683|  4832.197332264137|
| 2217.46915|  6694.331408987364|
|  1702.4553|-248.39998337383258|
| 1708.92575| 1421.3725393069162|
|  2304.0022| 1354.6579905880844|
| 17178.6824| 25918.614410103466|
|   1712.227|  2273.297295776685|
|  1720.3537|  2209.222936208973|
| 33732.6867| 26959.706039246157|
|    1727.54| 4063.7286166084305|
|   1727.785|-1943.7679331929012|
|   1737.376|  531.2976750771668|
| 2730.10785| 6914.7171942238065|
| 1632.03625|  979.0004407465094|
|22493.65964|  2781.594071963278|
+-----------+-------------------+
only showing top 20 rows



In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="charges", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2: {r2}")


Root Mean Squared Error (RMSE): 5660.558250981995
R^2: 0.7955176463005217


In [0]:
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print(f"Mean Absolute Error (MAE): {mae}")


Mean Absolute Error (MAE): 3960.005525917074


In [0]:
residuals = predictions.withColumn("residuals", F.col("charges") - F.col("prediction"))
residuals.select("residuals").describe().show()


+-------+-------------------+
|summary|          residuals|
+-------+-------------------+
|  count|                230|
|   mean|-181.90409400665038|
| stddev|  5669.974176858363|
|    min| -10351.14993283867|
|    max|  30453.47208040627|
+-------+-------------------+



In [0]:
dbutils.fs.unmount("/mnt/megacontainer1")


True

In [0]:
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point = "/mnt/megacontainer1",  
    extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token}
)


True